In [1]:
import pandas as pd # standard python data library
import geopandas as gp # the geo-version of pandas
import numpy as np 
import os
import fiona
from statistics import mean, median
from pandas import read_csv
gp.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw' #To load KML files
import string
import xml.etree.ElementTree as et

pd.options.display.max_columns = 999

# Utah

## VEST Documentation

Election results from OpenElections (https://github.com/openelections/openelections-data-ut/tree/master/2020).
Precinct shapefile from Utah Automated Geographic Reference Center (https://gis.utah.gov/data/political/voter-precincts/).

The shapefile from the AGRC is of subprecincts, breaking down precincts in cases of district splits. In some cases, results are reported at the subprecinct level, in most cases, they weren't, so merging was done where necessary.

Emery County - 52 "canvas votes" were distributed to precincts.

G20PRERTRU - Donald J. Trump (Republican Party)  
G20PREDBID - Joseph R. Biden (Democratic Party)  
G20PRELJOR - Jo Jorgensen (Libertarian Party)  
G20PREGHAW - Howie Hawkins (Green Party)  
G20PRECBLA - Don Blankenship (Constitution Party)  
G20PREIPIE - Brock Pierce (Unaffiliated)  
G20PREIWES - Kanye West (Unaffiliated)  
G20PREIMCH - Joe McHugh (Unaffiliated)  
G20PREILAR - Gloria La Riva (Unaffiliated)  
G20PREOWRI - Write-in Votes  

G20GOVRCOX - Spencer J. Cox (Republican Party)  
G20GOVDPET - Chris Peterson (Democratic Party)  
G20GOVLCOT - Daniel Rhead Cottam (Libertarian Party)  
G20GOVADUE - Greg Duerden (Independent American Party)  
G20GOVOWRI - Write-in Votes  

G20ATGRREY - Sean D. Reyes (Republican Party)  
G20ATGDSKO - Greg Skordas (Democratic Party)  
G20ATGLBAU - Rudy J. Bautista (Libertarian Party)  

G20AUDRDOU - John "Frugal" Dougall (Republican Party)  
G20AUDCOST - Jeffrey L. Ostler (Constitution Party)  
G20AUDUFAB - Brian L. Fabbi (United Utah Party)  

G20TRERDAM - David Damschen (Republican Party)  
G20TRELSPE - Joseph Speciale (Libertarian Party)  
G20TRECPRO - Richard Proctor (Constitution Party)  

## Load VEST File

In [2]:
vest_ut_20 = gp.read_file("./raw-from-source/VEST/ut_2020/ut_2020.shp")

In [3]:
#Create a list of data columns
data_columns = [col for col in vest_ut_20.columns if "G20" in col]

## Load Election Results

In [4]:
election_results = pd.read_csv("./raw-from-source/Election_Results/openelections-data-ut-master/2020/20201103__ut__general__precinct.csv")

/Users/peterhorton/opt/anaconda3/envs/geo_env/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
election_results["office"].unique()

array(['Registered Voters', 'Ballots Cast', 'Ballots Cast Blank',
       'President', 'U.S. House', 'Governor', 'Attorney General',
       'State Auditor', 'State Treasurer', 'State Senate', 'State House'],
      dtype=object)

In [6]:
election_results = election_results[election_results["office"].isin(['President','Governor', 'Attorney General','State Auditor', 'State Treasurer'])]

In [7]:
election_results["cand_detailed"] = election_results["office"]+"-"++election_results["candidate"].str.upper()

In [8]:
election_results["cand_detailed"] = election_results["cand_detailed"].astype(str)
print(list(election_results["cand_detailed"].unique()).sort())

None


In [9]:
name_list = list(election_results["cand_detailed"].unique())
name_list.sort()
name_list

['Attorney General-DEM GREG SKORDAS',
 'Attorney General-GREG SKORDAS',
 'Attorney General-LIB RUDY J. BAUTISTA',
 'Attorney General-REP SEAN D. REYES',
 'Attorney General-RUDY J. BAUTISTA',
 'Attorney General-SEAN D. REYES',
 'Governor-CHRIS PETERSEN',
 'Governor-CHRIS PETERSON',
 'Governor-CHRIS PETERSON / KARINA BROWN',
 'Governor-CHRIS PETERSON, KARINA BROWN',
 'Governor-DANIEL RHEAD COTTAM',
 'Governor-DANIEL RHEAD COTTAM / BARRY EVAN SHORT',
 'Governor-DANIEL RHEAD COTTAM, BARRY EVAN SHORT',
 'Governor-DEM CHRIS PETERSON',
 'Governor-GREG DUERDEN',
 'Governor-GREG DUERDEN / WAYNE HILL',
 'Governor-GREG DUERDEN, WAYNE HILL',
 'Governor-GREGORY C. JOHNSON (W)',
 'Governor-IAP GREG DUERDEN',
 'Governor-KRISTENA M. CONLIN',
 'Governor-KRISTENA M. CONLIN (W)',
 'Governor-LIB DANIEL RHEAD COTTAM',
 'Governor-MADELINE KAZANTZIS',
 'Governor-MADELINE KAZANTZIS / ED KENNEDY',
 'Governor-NOT ASSIGNED',
 'Governor-REP SPENCER J. COX',
 'Governor-RICHARD T. WHITNEY',
 'Governor-RICHARD T. WH

In [10]:
print(election_results[election_results["cand_detailed"].isna()])
print(election_results[election_results["cand_detailed"]=="nan"])

Empty DataFrame
Columns: [county, precinct, office, district, candidate, party, votes, mail, election_day, early_voting, cand_detailed]
Index: []
     county precinct    office  district candidate party votes  mail  \
1555   Iron       CH  Governor       NaN       NaN   NaN     1   NaN   

      election_day  early_voting cand_detailed  
1555           NaN           NaN           nan  


In [11]:
election_results = election_results[~election_results["cand_detailed"].isna()]
election_results = election_results[election_results["cand_detailed"]!="nan"]

In [12]:
cand_rename_dict = {
 #Presidential Candidates
 'President-REP DONALD J. TRUMP':"G20PRERTRU",
 'President-DONALD J. TRUMP':"G20PRERTRU",
 'President-DONALD J. TRUMP / MICHAEL R. PENCE':"G20PRERTRU",
 'President-DONALD J. TRUMP, MICHAEL R. PENCE':"G20PRERTRU",
    
 'President-JOSEPH R. BIDEN':"G20PREDBID",
 'President-JOSEPH R. BIDEN / KAMALA D. HARRIS':"G20PREDBID",
 'President-JOSEPH R. BIDEN, KAMALA D. HARRIS':"G20PREDBID",
 'President-DEM JOSEPH R. BIDEN':"G20PREDBID", 

 'President-JO JORGENSEN':"G20PRELJOR",
 'President-JO JORGENSEN / JEREMY COHEN':"G20PRELJOR",
 'President-JO JORGENSEN, JEREMY COHEN':"G20PRELJOR",
 'President-LIB JO JORGENSEN':"G20PRELJOR",
    
 'President-HOWIE HAWKINS':"G20PREGHAW",
 'President-HOWIE HAWKINS / ANGELA WALKER':"G20PREGHAW",
 'President-HOWIE HAWKINS, ANGELA WALKER':"G20PREGHAW",
 'President-GRN HOWIE HAWKINS':"G20PREGHAW",
    
 'President-CON DON BLANKENSHIP':"G20PRECBLA",
 'President-DON BALLOTS CAST BLANKENSHIP':"G20PRECBLA",
 'President-DON BLANKENSHIP':"G20PRECBLA",
 'President-DON BLANKENSHIP / WILLIAM MOHR':"G20PRECBLA",
 'President-DON BLANKENSHIP, WILLIAM MOHR':"G20PRECBLA",
    
 'President-BROCK PIERCE':"G20PREIPIE",
 'President-BROCK PIERCE / KARLA BALLARD':"G20PREIPIE",
 'President-BROCK PIERCE, KARLA BALLARD':"G20PREIPIE",
    
 'President-KANYE WEST':"G20PREIWES",
 'President-KANYE WEST / MICHELLE TIDBALL':"G20PREIWES",
 'President-KANYE WEST, MICHELLE TIDBALL':"G20PREIWES",
    
 'President-JOE MCHUGH':"G20PREIMCH",
 'President-JOE MCHUGH / ELIZABETH STORM':"G20PREIMCH",
 'President-JOE MCHUGH, ELIZABETH STORM':"G20PREIMCH",
    
 'President-GLORIA LA RIVA':"G20PREILAR",
 'President-GLORIA LARIVA':"G20PREILAR",
    
 'President-BRIAN CARROLL':"G20PREOWRI",
 'President-BRIAN CARROLL (W)':"G20PREOWRI",
 'President-WRITE-IN: BRIAN CARROLL':"G20PREOWRI",
 'President-WRITE-IN~ BRIAN CARROLL':"G20PREOWRI",
 'President-WRITE-IN~ JADE SIMMONS':"G20PREOWRI",
 'President-JADE SIMMONS (W)':"G20PREOWRI",
 'President-JADE SIMMONS / CLAUDELIAH J. ROZE':"G20PREOWRI",
 'President-WRITE-IN: JADE':"G20PREOWRI",
 'President-WRITE-IN: JADE SIMMONS':"G20PREOWRI",
 'President-KATHERINE FORBES':"G20PREOWRI",
 'President-PRESIDENT R. BODDIE (W)':"G20PREOWRI",
 'President-PRESIDENT R. BODDIE / ERIC C. STONEHAM':"G20PREOWRI",
 'President-PRINCESS KHADIJAH M. JACOB-FAMBRO (W)':"G20PREOWRI",
 'President-PRINCESS KHADIJAH M. JACOB-FAMBRO / KHADIJAH MARYAM JACOB SR.':"G20PREOWRI",
 'President-TOM HOEFLING':"G20PREOWRI",
 'President-TOM HOEFLING (W)':"G20PREOWRI",
 'President-WRITE-IN: TOM HOEFLING':"G20PREOWRI",
 'President-WRITE-IN':"G20PREOWRI",
 'President-WRITE-IN TOTALS':"G20PREOWRI",
 'President-WRITE-IN: UN-REGISTERED WRITE IN':"G20PREOWRI",
 'President-WRITE-INS':"G20PREOWRI",

 #EXPERIMENT HERE w/ including
#  'President-WRITE-IN: INVALID':"DROP",
#  'President-WRITE-IN: NOT CERTIFIED':"DROP",   
#  'President-NOT ASSIGNED':"DROP",
 'President-WRITE-IN: INVALID':"G20PREOWRI",
 'President-WRITE-IN: NOT CERTIFIED':"G20PREOWRI",   
 'President-NOT ASSIGNED':"G20PREOWRI",
    
 #Gubernatorial Candidates
 'Governor-CHRIS PETERSEN':"G20GOVDPET",
 'Governor-CHRIS PETERSON':"G20GOVDPET",
 'Governor-CHRIS PETERSON / KARINA BROWN':"G20GOVDPET",
 'Governor-CHRIS PETERSON, KARINA BROWN':"G20GOVDPET",
 'Governor-DEM CHRIS PETERSON':"G20GOVDPET",
    
 'Governor-DANIEL RHEAD COTTAM':"G20GOVLCOT",
 'Governor-DANIEL RHEAD COTTAM / BARRY EVAN SHORT':"G20GOVLCOT",
 'Governor-DANIEL RHEAD COTTAM, BARRY EVAN SHORT':"G20GOVLCOT",
 'Governor-LIB DANIEL RHEAD COTTAM':"G20GOVLCOT",
 
 'Governor-GREG DUERDEN':"G20GOVADUE",
 'Governor-GREG DUERDEN / WAYNE HILL':"G20GOVADUE",
 'Governor-GREG DUERDEN, WAYNE HILL':"G20GOVADUE",
 'Governor-IAP GREG DUERDEN':"G20GOVADUE",
    
 'Governor-REP SPENCER J. COX':"G20GOVRCOX",
 'Governor-SPENCER J. COX':"G20GOVRCOX",
 'Governor-SPENCER J. COX / DEIDRE M. HENDERSON':"G20GOVRCOX",
 'Governor-SPENCER J. COX, DEIDRE M. HENDERSON':"G20GOVRCOX",
    
 'Governor-GREGORY C. JOHNSON (W)':"G20GOVOWRI",

 'Governor-KRISTENA M. CONLIN':"G20GOVOWRI",
 'Governor-KRISTENA M. CONLIN (W)':"G20GOVOWRI",
 'Governor-WRITE-IN~ KRISTENA M CONLIN':"G20GOVOWRI",
 'Governor-WRITE-IN: KRISTENA M. CONLIN':"G20GOVOWRI", 
    
 'Governor-MADELINE KAZANTZIS':"G20GOVOWRI",
 'Governor-MADELINE KAZANTZIS / ED KENNEDY':"G20GOVOWRI",
 'Governor-WRITE-IN: MADELINE KAZANTZIS':"G20GOVOWRI",
 'Governor-WRITE-IN: MADELINE KAZANTZIS ED KENNEDY':"G20GOVOWRI",
 'Governor-WRITE-IN: MADELINE KAZANTZIS, ED':"G20GOVOWRI",
 'Governor-WRITE-IN: MADELINE KAZANTZIS/ED':"G20GOVOWRI",
 'Governor-WRITE-IN~ MADELINE KAZANTZIS':"G20GOVOWRI",
     
 'Governor-RICHARD T. WHITNEY':"G20GOVOWRI",
 'Governor-RICHARD T. WHITNEY (W)':"G20GOVOWRI",
 'Governor-WRITE-IN: RICHARD T. WHITNEY':"G20GOVOWRI",
 'Governor-WRITE-IN~ RICHARD T. WHITNEY':"G20GOVOWRI",

 'Governor-TYLER SCOTT BATTY (W)':"G20GOVOWRI",
 'Governor-TYLER SCOTT BATTY / GREGORY C. JOHNSON':"G20GOVOWRI",
 'Governor-WRITE-IN: TYLER SCOTT BAILEY':"G20GOVOWRI",
 
 'Governor-WRITE-IN':"G20GOVOWRI",
 'Governor-WRITE-IN TOTALS':"G20GOVOWRI",
 'Governor-WRITE-IN: UN-REGISTERED WRITE-IN':"G20GOVOWRI",
 'Governor-WRITE-INS':"G20GOVOWRI",

 #EXPERIMENT HERE w/ including
#  'Governor-NOT ASSIGNED':"DROP",
#  'Governor-WRITE-IN: INVALID':"DROP",
#  'Governor-WRITE-IN: NOT CERTIFIED':"DROP", 
 'Governor-NOT ASSIGNED':"G20GOVOWRI",
 'Governor-WRITE-IN: INVALID':"G20GOVOWRI",
 'Governor-WRITE-IN: NOT CERTIFIED':"G20GOVOWRI", 
    
 #Attorney General Candidates
 'Attorney General-DEM GREG SKORDAS':'G20ATGDSKO',
 'Attorney General-GREG SKORDAS':'G20ATGDSKO',
 'Attorney General-LIB RUDY J. BAUTISTA':'G20ATGLBAU',
 'Attorney General-REP SEAN D. REYES':'G20ATGRREY',
 'Attorney General-RUDY J. BAUTISTA':'G20ATGLBAU',
 'Attorney General-SEAN D. REYES':'G20ATGRREY',
    
 #Auditor Candidates
 'State Auditor-BRIAN L. FABBI':'G20AUDUFAB',
 'State Auditor-CON JEFFREY L. OSTLER':'G20AUDCOST',
 'State Auditor-JEFFREY L. OSTLER':'G20AUDCOST',
 'State Auditor-JOHN "FRUGAL" DOUGALL':'G20AUDRDOU',
 'State Auditor-JOHN "FRUGAL\' DOUGALL':'G20AUDRDOU',
 "State Auditor-JOHN 'FRUGAL' DOUGALL":'G20AUDRDOU',
 "State Auditor-REP JOHN 'FRUGAL' DOUGALL":'G20AUDRDOU',
 'State Auditor-UUP BRIAN L. FABBI':'G20AUDUFAB',

 #Treasurer Candidates
 'State Treasurer-CON RICHARD PROCTOR':'G20TRECPRO',
 'State Treasurer-DAVID DAMSCHEN':'G20TRERDAM',
 'State Treasurer-DAVID DAMSHEN':'G20TRERDAM',
 'State Treasurer-JOSEPH SPECIALE':'G20TRELSPE',
 'State Treasurer-LIB JOSEPH SPECIALE':'G20TRELSPE',
 'State Treasurer-REP DAVID DAMSCHEN':'G20TRERDAM',
 'State Treasurer-RICHARD PROCTOR':'G20TRECPRO', 
}

In [13]:
election_results["cand_detailed"] = election_results["cand_detailed"].map(cand_rename_dict).fillna(election_results["cand_detailed"])

In [14]:
election_results = election_results[election_results["cand_detailed"]!="DROP"]

In [15]:
election_results["cand_detailed"].unique()

array(['G20PREIPIE', 'G20PREIWES', 'G20PREDBID', 'G20PRECBLA',
       'G20PRELJOR', 'G20PREIMCH', 'G20PREGHAW', 'G20PREILAR',
       'G20PRERTRU', 'G20PREOWRI', 'G20GOVDPET', 'G20GOVADUE',
       'G20GOVRCOX', 'G20GOVLCOT', 'G20GOVOWRI', 'G20ATGLBAU',
       'G20ATGDSKO', 'G20ATGRREY', 'G20AUDRDOU', 'G20AUDCOST',
       'G20AUDUFAB', 'G20TRECPRO', 'G20TRERDAM', 'G20TRELSPE'],
      dtype=object)

In [16]:
election_results["pivot_col"] = election_results["county"]+"-"+election_results["precinct"]

In [17]:
election_results["votes"] = election_results["votes"].astype(str)
election_results["votes"] = election_results["votes"].str.replace(",","")
election_results["votes"] = election_results["votes"].astype(int)

In [18]:
election_results_pivot = pd.pivot_table(election_results,values="votes",index=["pivot_col","county","precinct"],columns=["cand_detailed"],aggfunc=sum)




In [19]:
election_results_pivot

,,cand_detailed,G20ATGDSKO,G20ATGLBAU,G20ATGRREY,G20AUDCOST,G20AUDRDOU,G20AUDUFAB,G20GOVADUE,G20GOVDPET,G20GOVLCOT,G20GOVOWRI,G20GOVRCOX,G20PRECBLA,G20PREDBID,G20PREGHAW,G20PREILAR,G20PREIMCH,G20PREIPIE,G20PREIWES,G20PRELJOR,G20PREOWRI,G20PRERTRU,G20TRECPRO,G20TRELSPE,G20TRERDAM
pivot_col,county,precinct,,,,,,,,,,,,,,,,,,,,,,,,
Beaver-BEAVER 1,Beaver,BEAVER 1,56.0,18.0,629.0,51.0,603.0,15.0,11.0,40.0,15.0,12.0,621.0,0.0,59.0,0.0,1.0,0.0,0.0,1.0,2.0,0.0,682.0,41.0,20.0,609.0
Beaver-BEAVER 2,Beaver,BEAVER 2,63.0,21.0,426.0,42.0,414.0,26.0,13.0,46.0,19.0,8.0,431.0,1.0,57.0,2.0,1.0,0.0,0.0,3.0,5.0,0.0,463.0,44.0,34.0,412.0
Beaver-BEAVER 3,Beaver,BEAVER 3,31.0,18.0,379.0,25.0,361.0,5.0,5.0,32.0,18.0,7.0,365.0,0.0,34.0,1.0,1.0,1.0,0.0,0.0,2.0,0.0,413.0,17.0,14.0,369.0
Beaver-BEAVER 4,Beaver,BEAVER 4,21.0,5.0,148.0,18.0,145.0,1.0,2.0,18.0,7.0,5.0,142.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,156.0,21.0,5.0,138.0
Beaver-GREENVILLE 1,Beaver,GREENVILLE 1,12.0,3.0,81.0,5.0,81.0,3.0,1.0,9.0,2.0,1.0,82.0,0.0,9.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,90.0,1.0,5.0,88.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Weber-West Haven 004,Weber,West Haven 004,554.0,151.0,1213.0,274.0,1337.0,191.0,29.0,555.0,61.0,NaN,1252.0,3.0,652.0,9.0,1.0,4.0,2.0,11.0,52.0,NaN,1245.0,189.0,301.0,1326.0
Weber-West Haven 005,Weber,West Haven 005,97.0,16.0,265.0,38.0,285.0,23.0,7.0,81.0,4.0,NaN,279.0,0.0,100.0,0.0,0.0,0.0,1.0,1.0,5.0,NaN,277.0,24.0,27.0,300.0
Weber-West Haven 006,Weber,West Haven 006,61.0,13.0,397.0,26.0,395.0,22.0,8.0,54.0,11.0,NaN,387.0,1.0,71.0,0.0,0.0,2.0,0.0,2.0,10.0,NaN,386.0,25.0,33.0,391.0


In [20]:
election_results_pivot.reset_index(inplace=True,drop=False)
election_results_pivot.reset_index(inplace=True,drop=True)

In [21]:
election_results_pivot

cand_detailed,pivot_col,county,precinct,G20ATGDSKO,G20ATGLBAU,G20ATGRREY,G20AUDCOST,G20AUDRDOU,G20AUDUFAB,G20GOVADUE,G20GOVDPET,G20GOVLCOT,G20GOVOWRI,G20GOVRCOX,G20PRECBLA,G20PREDBID,G20PREGHAW,G20PREILAR,G20PREIMCH,G20PREIPIE,G20PREIWES,G20PRELJOR,G20PREOWRI,G20PRERTRU,G20TRECPRO,G20TRELSPE,G20TRERDAM
0,Beaver-BEAVER 1,Beaver,BEAVER 1,56.0,18.0,629.0,51.0,603.0,15.0,11.0,40.0,15.0,12.0,621.0,0.0,59.0,0.0,1.0,0.0,0.0,1.0,2.0,0.0,682.0,41.0,20.0,609.0
1,Beaver-BEAVER 2,Beaver,BEAVER 2,63.0,21.0,426.0,42.0,414.0,26.0,13.0,46.0,19.0,8.0,431.0,1.0,57.0,2.0,1.0,0.0,0.0,3.0,5.0,0.0,463.0,44.0,34.0,412.0
2,Beaver-BEAVER 3,Beaver,BEAVER 3,31.0,18.0,379.0,25.0,361.0,5.0,5.0,32.0,18.0,7.0,365.0,0.0,34.0,1.0,1.0,1.0,0.0,0.0,2.0,0.0,413.0,17.0,14.0,369.0
3,Beaver-BEAVER 4,Beaver,BEAVER 4,21.0,5.0,148.0,18.0,145.0,1.0,2.0,18.0,7.0,5.0,142.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,156.0,21.0,5.0,138.0
4,Beaver-GREENVILLE 1,Beaver,GREENVILLE 1,12.0,3.0,81.0,5.0,81.0,3.0,1.0,9.0,2.0,1.0,82.0,0.0,9.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,90.0,1.0,5.0,88.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2341,Weber-West Haven 004,Weber,West Haven 004,554.0,151.0,1213.0,274.0,1337.0,191.0,29.0,555.0,61.0,NaN,1252.0,3.0,652.0,9.0,1.0,4.0,2.0,11.0,52.0,NaN,1245.0,189.0,301.0,1326.0
2342,Weber-West Haven 005,Weber,West Haven 005,97.0,16.0,265.0,38.0,285.0,23.0,7.0,81.0,4.0,NaN,279.0,0.0,100.0,0.0,0.0,0.0,1.0,1.0,5.0,NaN,277.0,24.0,27.0,300.0
2343,Weber-West Haven 006,Weber,West Haven 006,61.0,13.0,397.0,26.0,395.0,22.0,8.0,54.0,11.0,NaN,387.0,1.0,71.0,0.0,0.0,2.0,0.0,2.0,10.0,NaN,386.0,25.0,33.0,391.0
2344,Weber-West Haven 007,Weber,West Haven 007,194.0,38.0,592.0,92.0,617.0,66.0,9.0,172.0,25.0,NaN,594.0,4.0,214.0,3.0,0.0,1.0,1.0,7.0,20.0,NaN,595.0,75.0,98.0,604.0


In [22]:
def statewide_totals_check(partner_df,source_df,column_list):
    print("***Statewide Totals Check***")
    for race in column_list:
        if (partner_df[race].sum()- source_df[race].sum() != 0):
            print(race+" has a difference of "+str(partner_df[race].sum()-source_df[race].sum())+" votes")
            print("\tVEST: "+str(partner_df[race].sum())+" votes")
            print("\tSOURCES: "+str(source_df[race].sum())+" votes")
        else:
            print(race + " is equal", "\tVEST / RDH: " + str(partner_df[race].sum()))

In [23]:
statewide_totals_check(vest_ut_20,election_results_pivot,data_columns)

***Statewide Totals Check***
G20PRERTRU has a difference of 1.0 votes
	VEST: 865140 votes
	SOURCES: 865139.0 votes
G20PREDBID is equal 	VEST / RDH: 560282
G20PRELJOR is equal 	VEST / RDH: 38447
G20PREGHAW is equal 	VEST / RDH: 5053
G20PRECBLA is equal 	VEST / RDH: 5551
G20PREIPIE is equal 	VEST / RDH: 2623
G20PREIWES is equal 	VEST / RDH: 7213
G20PREIMCH is equal 	VEST / RDH: 2229
G20PREILAR is equal 	VEST / RDH: 1139
G20PREOWRI is equal 	VEST / RDH: 18965
G20GOVRCOX has a difference of 1.0 votes
	VEST: 918754 votes
	SOURCES: 918753.0 votes
G20GOVDPET is equal 	VEST / RDH: 442754
G20GOVLCOT is equal 	VEST / RDH: 51393
G20GOVADUE is equal 	VEST / RDH: 25810
G20GOVOWRI has a difference of 1.0 votes
	VEST: 38170 votes
	SOURCES: 38169.0 votes
G20ATGRREY has a difference of 1.0 votes
	VEST: 878853 votes
	SOURCES: 878852.0 votes
G20ATGDSKO is equal 	VEST / RDH: 489500
G20ATGLBAU is equal 	VEST / RDH: 82444
G20AUDRDOU has a difference of 1.0 votes
	VEST: 1000846 votes
	SOURCES: 1000845.0 vote

In [24]:
vest_ut_20["CountyID"].value_counts()

vest_ut_20["CountyID"].value_counts().index.values

array([18,  6, 25, 29, 27,  3, 23,  2, 22, 11, 26,  7, 10, 24, 20, 21, 19,
        4, 12, 14, 15,  8, 13,  9,  1, 28, 17, 16,  5])

In [25]:
election_results_pivot["county"].value_counts()
election_results_pivot["county"].value_counts().index.values

array(['Salt Lake', 'Davis', 'Utah', 'Weber', 'Washington', 'Cache',
       'Tooele', 'Box Elder', 'Summit', 'Iron', 'Duchesne', 'Wasatch',
       'Grand', 'Uintah', 'Sanpete', 'Sevier', 'San Juan', 'Carbon',
       'Juab', 'Millard', 'Morgan', 'Emery', 'Kane', 'Garfield', 'Beaver',
       'Wayne', 'Piute', 'Daggett', 'Rich'], dtype=object)

In [26]:
county_id_dict = dict(zip(vest_ut_20["CountyID"].value_counts().index.values,election_results_pivot["county"].value_counts().index.values))

In [27]:
{k: v for k, v in sorted(county_id_dict.items(), key=lambda item: item[1])}

{1: 'Beaver',
 2: 'Box Elder',
 3: 'Cache',
 4: 'Carbon',
 16: 'Daggett',
 6: 'Davis',
 26: 'Duchesne',
 8: 'Emery',
 9: 'Garfield',
 10: 'Grand',
 11: 'Iron',
 12: 'Juab',
 13: 'Kane',
 14: 'Millard',
 15: 'Morgan',
 17: 'Piute',
 5: 'Rich',
 18: 'Salt Lake',
 19: 'San Juan',
 20: 'Sanpete',
 21: 'Sevier',
 22: 'Summit',
 23: 'Tooele',
 24: 'Uintah',
 25: 'Utah',
 7: 'Wasatch',
 27: 'Washington',
 28: 'Wayne',
 29: 'Weber'}

In [28]:
county_id_dict ={1: 'Beaver',
 2: 'Box Elder',
 3: 'Cache',
 4: 'Carbon',
 5: 'Daggett',
 6: 'Davis',
 7: 'Duchesne',
 8: 'Emery',
 9: 'Garfield',
 10: 'Grand',
 11: 'Iron',
 12: 'Juab',
 13: 'Kane',
 14: 'Millard',
 15: 'Morgan',
 16: 'Piute',
 17: 'Rich',
 18: 'Salt Lake',
 19: 'San Juan',
 20: 'Sanpete',
 21: 'Sevier',
 22: 'Summit',
 23: 'Tooele',
 24: 'Uintah',
 25: 'Utah',
 26: 'Wasatch',
 27: 'Washington',
 28: 'Wayne',
 29: 'Weber'}

In [29]:
county_id_dict_2 = {}
for val in county_id_dict:
    county_id_dict_2[county_id_dict[val]]=val 

In [30]:
county_id_dict_2

{'Beaver': 1,
 'Box Elder': 2,
 'Cache': 3,
 'Carbon': 4,
 'Daggett': 5,
 'Davis': 6,
 'Duchesne': 7,
 'Emery': 8,
 'Garfield': 9,
 'Grand': 10,
 'Iron': 11,
 'Juab': 12,
 'Kane': 13,
 'Millard': 14,
 'Morgan': 15,
 'Piute': 16,
 'Rich': 17,
 'Salt Lake': 18,
 'San Juan': 19,
 'Sanpete': 20,
 'Sevier': 21,
 'Summit': 22,
 'Tooele': 23,
 'Uintah': 24,
 'Utah': 25,
 'Wasatch': 26,
 'Washington': 27,
 'Wayne': 28,
 'Weber': 29}

In [31]:
election_results_pivot["CountyID"] = election_results_pivot["county"].map(county_id_dict_2)

In [32]:
election_results_pivot["CountyID"].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29])

In [33]:
def county_totals_check(partner_df,source_df,column_list,county_col,full_print=False):
    print("***Countywide Totals Check***")
    print("")
    diff_counties=[]
    for race in column_list:
        diff = partner_df.groupby([county_col]).sum()[race]-source_df.groupby([county_col]).sum()[race]
        for val in diff[diff != 0].index.values.tolist():
            if val not in diff_counties:
                diff_counties.append(val)
        if len(diff[diff != 0]!=0):   
            print(race + " contains differences in these counties:")
            for val in diff[diff != 0].index.values.tolist():
                county_differences = diff[diff != 0]
                print("\t"+val+" has a difference of "+str(county_differences[val])+" votes")
                print("\t\tVEST: "+str(partner_df.groupby([county_col]).sum().loc[val,race])+" votes")
                print("\t\tSOURCES: "+str(source_df.groupby([county_col]).sum().loc[val,race])+" votes")
            if (full_print):
                for val in diff[diff == 0].index.values.tolist():
                    county_similarities = diff[diff == 0]
                    print("\t"+val + ": "+ str(partner_df.groupby([county_col]).sum().loc[val,race])+" votes")
        else:
            print(race + " is equal across all counties")
            if (full_print):
                for val in diff[diff == 0].index.values.tolist():
                    county_similarities = diff[diff == 0]
                    print("\t"+val + ": "+ str(partner_df.groupby([county_col]).sum().loc[val,race])+" votes")

In [34]:
election_results_pivot =election_results_pivot.fillna(0)

In [35]:
election_results_pivot["CountyID"] = election_results_pivot["CountyID"].astype(str)
vest_ut_20["CountyID"] = vest_ut_20["CountyID"].astype(str)

In [36]:
for val in data_columns:
    election_results_pivot[val]=election_results_pivot[val].astype(int)
    vest_ut_20[val]=vest_ut_20[val].astype(int)

In [37]:
county_totals_check(vest_ut_20,election_results_pivot,data_columns,"CountyID",full_print=False)

***Countywide Totals Check***

G20PRERTRU contains differences in these counties:
	7 has a difference of 1 votes
		VEST: 7513 votes
		SOURCES: 7512 votes
G20PREDBID is equal across all counties
G20PRELJOR is equal across all counties
G20PREGHAW is equal across all counties
G20PRECBLA is equal across all counties
G20PREIPIE is equal across all counties
G20PREIWES is equal across all counties
G20PREIMCH is equal across all counties
G20PREILAR is equal across all counties
G20PREOWRI is equal across all counties
G20GOVRCOX contains differences in these counties:
	7 has a difference of 1 votes
		VEST: 6096 votes
		SOURCES: 6095 votes
G20GOVDPET is equal across all counties
G20GOVLCOT is equal across all counties
G20GOVADUE is equal across all counties
G20GOVOWRI contains differences in these counties:
	11 has a difference of 1 votes
		VEST: 1603 votes
		SOURCES: 1602 votes
G20ATGRREY contains differences in these counties:
	7 has a difference of 1 votes
		VEST: 7054 votes
		SOURCES: 7053 vo

In [38]:
vest_ut_20["unique_ID"] = vest_ut_20["CountyID"]+"-"+vest_ut_20["resultspre"]
vest_ut_20["unique_ID"].value_counts(dropna=False)

NaN                   83
18-SLC012              1
2-Perry 4              1
10-Moab 05-570         1
18-MUR039              1
                      ..
18-SAN063              1
27-COHS74              1
29-Ogden 007           1
2-Brigham City 1       1
29-North Ogden 005     1
Name: unique_ID, Length: 2342, dtype: int64

In [39]:
for i in data_columns:
    print(i)
    print(sum(vest_ut_20[vest_ut_20["unique_ID"].isna()][i]))

G20PRERTRU
0
G20PREDBID
0
G20PRELJOR
0
G20PREGHAW
0
G20PRECBLA
0
G20PREIPIE
0
G20PREIWES
0
G20PREIMCH
0
G20PREILAR
0
G20PREOWRI
0
G20GOVRCOX
0
G20GOVDPET
0
G20GOVLCOT
0
G20GOVADUE
0
G20GOVOWRI
0
G20ATGRREY
0
G20ATGDSKO
0
G20ATGLBAU
0
G20AUDRDOU
0
G20AUDCOST
0
G20AUDUFAB
0
G20TRERDAM
0
G20TRELSPE
0
G20TRECPRO
0


In [40]:
election_results_pivot["unique_ID"] = election_results_pivot["CountyID"]+"-"+election_results_pivot["precinct"]

In [41]:
election_results_pivot["unique_ID"].value_counts(dropna=False)

10-Moab 04-570         1
6-Kaysville 17:U-C-    1
18-SAN001              1
26-43                  1
6-Bountiful 23:I-S-    1
                      ..
25-Cedar Hills 03      1
3-NIBLEY 03            1
18-MAG011              1
18-WVC052              1
29-North Ogden 005     1
Name: unique_ID, Length: 2346, dtype: int64

In [42]:
def allocate_absentee(df_receiving_votes,df_allocating,column_list,col_allocating):
    original_cols = list(df_receiving_votes.columns)
    
    #Add in the "Total Votes column"
    df_receiving_votes.loc[:,"Total_Votes"]=0
    for race in column_list:
        df_receiving_votes.loc[:,"Total_Votes"]+=df_receiving_votes.loc[:,race]
    
    #Create the needed dataframes
    precinct_specific_totals = pd.DataFrame(df_receiving_votes.groupby([col_allocating]).sum())
    precinct_specific_totals.reset_index(drop=False,inplace=True)
    to_dole_out_totals = pd.DataFrame(df_allocating.groupby([col_allocating]).sum())
    to_dole_out_totals.reset_index(drop=False,inplace=True)
    
    #Print out any instances where the allocation, as written, won't work
    special_allocation_needed = []
    for index, row in precinct_specific_totals.iterrows():
        for race in column_list:
            if (row[race]==0):
                race_district = row[col_allocating]
                if race_district in to_dole_out_totals[col_allocating].unique():
                    to_allocate = int(to_dole_out_totals.loc[to_dole_out_totals[col_allocating]==race_district][race])
                    if (to_allocate != 0):
                        special_allocation_needed.append([race_district,race])
    
    #Create some new columns for each of these races to deal with the allocation
    for race in column_list:
        add_var = race+"_add"
        rem_var = race+"_rem"
        floor_var = race+"_floor"
        df_receiving_votes.loc[:,add_var]=0.0
        df_receiving_votes.loc[:,rem_var]=0.0
        df_receiving_votes.loc[:,floor_var]=0.0

    #Iterate over the rows
    #Note this function iterates over the dataframe two times so the rounded vote totals match the totals to allocate
    for index, row in df_receiving_votes.iterrows():
        if row[col_allocating] in to_dole_out_totals[col_allocating].unique():
            for race in column_list:
                add_var = race+"_add"
                rem_var = race+"_rem"
                floor_var = race+"_floor"
                #Grab the district
                county_id = row[col_allocating]
                if [county_id,race] in special_allocation_needed:
                    #Get the denominator for the allocation - the summed "total votes" for precincts in that grouping
                    denom = precinct_specific_totals.loc[precinct_specific_totals[col_allocating]==county_id]["Total_Votes"]
                    #Get one of the numerators, how many districtwide votes to allocate
                    numer = to_dole_out_totals.loc[to_dole_out_totals[col_allocating]==county_id][race]
                    #Get the "total votes" for this particular precinct
                    val = df_receiving_votes.at[index,"Total_Votes"]
                    #Get the vote share, the precincts % of total precinct votes in the district times votes to allocate
                else:
                    #Get the denominator for the allocation (the precinct vote totals)
                    denom = precinct_specific_totals.loc[precinct_specific_totals[col_allocating]==county_id][race]
                    #Get one of the numerators, how many districtwide votes to allocate
                    numer = to_dole_out_totals.loc[to_dole_out_totals[col_allocating]==county_id][race]
                    #Get the vote totals for this race in this precinct
                    val = df_receiving_votes.at[index,race]
                    #Get the vote share, the precincts % of total precinct votes in the district times votes to allocate
                if ((float(denom)==0)):
                    vote_share = 0
                else:
                    vote_share = (float(val)/float(denom))*float(numer)
                df_receiving_votes.at[index,add_var] = vote_share
                #Take the decimal remainder of the allocation
                df_receiving_votes.at[index,rem_var] = vote_share%1
                #Take the floor of the allocation
                df_receiving_votes.at[index,floor_var] = np.floor(vote_share)

    #After the first pass through, get the sums of the races by district to assist in the rounding            
    first_allocation = pd.DataFrame(df_receiving_votes.groupby([col_allocating]).sum())

    #Now we want to iterate district by district to work on rounding
    county_list = list(to_dole_out_totals[col_allocating].unique()) 

    #Iterate over the district
    for county in county_list:
        for race in column_list:
            add_var = race+"_add"
            rem_var = race+"_rem"
            floor_var = race+"_floor"
            #County how many votes still need to be allocated (because we took the floor of all the initial allocations)
            to_go = int(np.round((int(to_dole_out_totals.loc[to_dole_out_totals[col_allocating]==county][race])-first_allocation.loc[first_allocation.index==county,floor_var])))
            #Grab the n precincts with the highest remainders and round these up, where n is the # of votes that still need to be allocated
            for index in df_receiving_votes.loc[df_receiving_votes[col_allocating]==county][rem_var].nlargest(to_go).index:
                df_receiving_votes.at[index,add_var] = np.ceil(df_receiving_votes.at[index,add_var])

    #Iterate over every race again
    for race in column_list:
        add_var = race+"_add"
        #Round every allocation down to not add fractional votes
        df_receiving_votes.loc[:,add_var]=np.floor(df_receiving_votes.loc[:,add_var])
        df_receiving_votes.loc[:,race]+=df_receiving_votes.loc[:,add_var]
        df_receiving_votes.loc[:,race] = df_receiving_votes.loc[:,race].astype(int)
        
    df_receiving_votes = df_receiving_votes[original_cols]
    
    return df_receiving_votes

In [43]:
#8-Canvas need to be distributed

In [44]:
to_allocate = election_results_pivot[election_results_pivot["unique_ID"]=="8-Canvas"]
receiving_votes = election_results_pivot[election_results_pivot["unique_ID"]!="8-Canvas"]

In [45]:
election_results = allocate_absentee(receiving_votes,to_allocate,data_columns,"CountyID")

/Users/peterhorton/opt/anaconda3/envs/geo_env/lib/python3.9/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/Users/peterhorton/opt/anaconda3/envs/geo_env/lib/python3.9/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/Users/peterhorton/opt/anaconda3/envs/geo_env/lib/python3.9/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying

In [46]:
join_attempt_one = pd.merge(vest_ut_20,election_results,how="outer",on="unique_ID",indicator=True)
print(join_attempt_one["_merge"].value_counts())

both          2319
left_only      105
right_only      26
Name: _merge, dtype: int64


In [47]:
join_attempt_one[join_attempt_one["_merge"]=="left_only"].to_csv("./vest_only.csv")
join_attempt_one[join_attempt_one["_merge"]=="right_only"].to_csv("./source_only.csv")

In [48]:
id_changes_dict = {
'21-Monroe 1':'21-Monroe 8',
'21-Monroe 2':'21-Monroe 9',
'24-Vernal City 1':'24-Vernal City 8',
'21-Richfield 7':'21-Richfield 23',
'24-Vernal City 3':'24-Vernal City 10',
'21-Richfield 8':'21-Richfield 24',
'24-Vernal City 2':'24-Vernal City 9',
'24-Vernal City 5':'24-Vernal City 12',
'24-Vernal City 6':'24-Vernal City 13',
'21-Richfield 3':'21-Richfield 13',
'21-Salina 2':'21-Salina 15',
'21-Richfield 1':'21-Richfield 11',
'21-Richfield 6':'21-Richfield 22',
'21-Richfield 2':'21-Richfield 12',
'24-Vernal City 4':'24-Vernal City 11',
'21-Richfield 4':'21-Richfield 20',
'21-Richfield 5':'21-Richfield 21',
'21-Salina 1':'21-Salina 14',
'21-Salina 3':'21-Salina 25',
'16-Marysville':'16-Marysvale',
'24-Vernal City 7':'24-Vernal City 14',
'23-G-17:0':'23-LP-17:0',
}

In [49]:
election_results["unique_ID"] = election_results["unique_ID"].map(id_changes_dict).fillna(election_results_pivot["unique_ID"])

<ipython-input-49-4adf7e5a08ca>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  election_results["unique_ID"] = election_results["unique_ID"].map(id_changes_dict).fillna(election_results_pivot["unique_ID"])


In [50]:
join_attempt_two = pd.merge(vest_ut_20,election_results,how="outer",on="unique_ID",indicator=True)
print(join_attempt_two["_merge"].value_counts())

both          2341
left_only       83
right_only       4
Name: _merge, dtype: int64


In [51]:
for i in data_columns:
    left = i+"_x"
    right = i+"_y"
    print(i)
    print(sum(join_attempt_two[join_attempt_two["_merge"]=="left_only"][left]))
    print(sum(join_attempt_two[join_attempt_two["_merge"]=="right_only"][right]))

G20PRERTRU
0.0
0.0
G20PREDBID
0.0
0.0
G20PRELJOR
0.0
0.0
G20PREGHAW
0.0
0.0
G20PRECBLA
0.0
0.0
G20PREIPIE
0.0
0.0
G20PREIWES
0.0
0.0
G20PREIMCH
0.0
0.0
G20PREILAR
0.0
0.0
G20PREOWRI
0.0
0.0
G20GOVRCOX
0.0
0.0
G20GOVDPET
0.0
0.0
G20GOVLCOT
0.0
0.0
G20GOVADUE
0.0
0.0
G20GOVOWRI
0.0
0.0
G20ATGRREY
0.0
0.0
G20ATGDSKO
0.0
0.0
G20ATGLBAU
0.0
0.0
G20AUDRDOU
0.0
0.0
G20AUDCOST
0.0
0.0
G20AUDUFAB
0.0
0.0
G20TRERDAM
0.0
0.0
G20TRELSPE
0.0
0.0
G20TRECPRO
0.0
0.0


In [52]:
def precinct_votes_check(merged_df,column_list,vest_on_left,name_col,print_level=0):
    merged_df = merged_df.sort_values(by=[name_col],inplace=False)
    matching_rows = 0
    different_rows = 0
    diff_list=[]
    diff_values = []
    max_diff = 0
    for index,row in merged_df.iterrows():
        same = True
        for i in column_list:
            left_data = i + "_x"
            right_data = i + "_y"
            if ((row[left_data] is None) or (row[right_data] is None) or (np.isnan(row[right_data])or(np.isnan(row[left_data])))):
                print("FIX NaN value at: ", row[name_col])
                return;
            diff = abs(row[left_data]-row[right_data])
            if (diff>0):
                same = False
                diff_values.append(abs(diff))
                if (diff>max_diff):
                    max_diff = diff
            if(diff>print_level):
                if (vest_on_left):
                    print(i, "{:.>72}".format(row[name_col]), "(V)","{:.>5}".format(int(row[left_data]))," (S){:.>5}".format(int(row[right_data])),"(D):{:>5}".format(int(row[left_data]-row[right_data])))                           
                else:
                    print(i, "{:.>72}".format(row[name_col]), "(S)","{:.>5}".format(int(row[left_data]))," (V){:.>5}".format(int(row[right_data])),"(D):{:>5}".format(int(row[left_data]-row[right_data])))
        if(same != True):
            different_rows +=1
            diff_list.append(row[name_col])
        else:
            matching_rows +=1
    print("")
    print("There are ", len(merged_df.index)," total rows")
    print(different_rows," of these rows have election result differences")
    print(matching_rows," of these rows are the same")
    print("")
    print("The max difference between any one shared column in a row is: ", max_diff)
    if(len(diff_values)!=0):
        print("The average difference is: ", str(sum(diff_values)/len(diff_values)))
    count_big_diff = len([i for i in diff_values if i > 10])
    print("There are ", str(count_big_diff), "precinct results with a difference greater than 10")
    print("")
    print("All precincts containing differences:")
    diff_list.sort()
    print(diff_list)

In [53]:
precinct_votes_check(join_attempt_two[join_attempt_two["_merge"]=="both"],data_columns,True,"unique_ID",print_level=0)

G20GOVOWRI ...................................................................11-CH (V) ....9  (S)....8 (D):    1
G20PRERTRU ...........................................7-AL11UC - UPPER COMM REC SSD (V) ..308  (S)..307 (D):    1
G20GOVRCOX ...........................................7-AL11UC - UPPER COMM REC SSD (V) ..240  (S)..239 (D):    1
G20ATGRREY ...........................................7-AL11UC - UPPER COMM REC SSD (V) ..292  (S)..291 (D):    1
G20AUDRDOU ...........................................7-AL11UC - UPPER COMM REC SSD (V) ..275  (S)..274 (D):    1
G20TRERDAM ...........................................7-AL11UC - UPPER COMM REC SSD (V) ..282  (S)..281 (D):    1
G20PRELJOR .............................................................8-Ferron #2 (V) ....5  (S)....6 (D):   -1
G20AUDUFAB .........................................................8-Huntington #9 (V) ...15  (S)...16 (D):   -1
G20PRELJOR ........................................................8-Orangeville #5 (V) 

## Join to Shapefile

In [54]:
ut_shapes = gp.read_file("./raw-from-source/Shapefiles/Utah_Vista_Ballot_Areas/VistaBallotAreas.shp")

In [55]:
# election_results.to_csv("./election_results.csv")
# ut_shapes.to_csv("./shapefile.csv")

In [56]:
election_results[election_results["precinct"].str.contains(":")]

cand_detailed,pivot_col,county,precinct,G20ATGDSKO,G20ATGLBAU,G20ATGRREY,G20AUDCOST,G20AUDRDOU,G20AUDUFAB,G20GOVADUE,G20GOVDPET,G20GOVLCOT,G20GOVOWRI,G20GOVRCOX,G20PRECBLA,G20PREDBID,G20PREGHAW,G20PREILAR,G20PREIMCH,G20PREIPIE,G20PREIWES,G20PRELJOR,G20PREOWRI,G20PRERTRU,G20TRECPRO,G20TRELSPE,G20TRERDAM,CountyID,unique_ID
178,Davis-Bountiful 01:I-S-,Davis,Bountiful 01:I-S-,292,58,464,120,548,110,26,243,15,15,524,3,341,0,0,2,2,4,28,19,433,90,107,571,6,6-Bountiful 01:I-S-
179,Davis-Bountiful 02:I-S-,Davis,Bountiful 02:I-S-,263,58,409,98,474,110,24,231,38,12,432,4,314,4,0,1,0,3,18,10,398,89,102,484,6,6-Bountiful 02:I-S-
180,Davis-Bountiful 03:I-S-,Davis,Bountiful 03:I-S-,214,30,372,84,417,70,9,179,18,7,412,3,249,2,0,1,2,1,21,29,327,62,75,434,6,6-Bountiful 03:I-S-
181,Davis-Bountiful 04:I---,Davis,Bountiful 04:I---,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,6,6-Bountiful 04:I---
182,Davis-Bountiful 04:I-S-,Davis,Bountiful 04:I-S-,252,39,432,76,498,104,16,195,27,8,487,4,259,2,1,0,2,6,26,21,413,61,95,514,6,6-Bountiful 04:I-S-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1683,Tooele-STK-02:1-02:2,Tooele,STK-02:1-02:2,111,37,522,63,553,36,30,79,39,0,502,2,107,0,0,2,0,3,13,0,558,68,59,522,23,23-STK-02:1-02:2
1686,Tooele-T-03:1-03:2,Tooele,T-03:1-03:2,196,17,333,69,404,29,5,147,18,0,362,4,211,1,0,1,0,4,20,0,314,65,52,382,23,23-T-03:1-03:2
1687,Tooele-T-04:1-04:2,Tooele,T-04:1-04:2,273,46,548,101,641,78,20,214,26,0,593,0,257,4,0,0,1,5,15,0,589,107,89,617,23,23-T-04:1-04:2
1706,Tooele-VERN-01:1-01:2,Tooele,VERN-01:1-01:2,18,9,114,13,114,7,7,19,7,0,95,0,16,0,0,0,0,1,1,0,121,22,15,98,23,23-VERN-01:1-01:2


In [57]:
whole_results = election_results[~election_results["precinct"].str.contains(":")]
subprecinct_results = election_results[election_results["precinct"].str.contains(":")]

In [58]:
election_results["shp_ID"] = election_results["unique_ID"]

<ipython-input-58-a5009c41849a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  election_results["shp_ID"] = election_results["unique_ID"]


In [59]:
ut_shapes["CountyID"] = ut_shapes["CountyID"].astype(str)
ut_shapes["shp_ID"] = ut_shapes["CountyID"]+"-"+ut_shapes["PrecinctID"]

In [60]:
no_dissolve_list = ['10-01',
 '10-02',
 '10-03',
 '10-04',
 '10-05',
 '10-06',
 '10-07',
 '10-08',
 '10-09',
 '10-10',
 '10-11',
 '12-EU',
 '12-NE01',
 '12-NE02',
 '12-NE03',
 '13-CW',
 '16-1',
 '16-2',
 '16-3',
 '16-4',
 '16-5',
 '17-UNINH',
 '18-DRP005',
 '18-MIL047',
 '18-MIL048',
 '18-NLC001',
 '18-NRV001',
 '18-NRV002',
 '18-NRV003',
 '18-NRV004',
 '18-NRV005',
 '18-NRV006',
 '18-NRV007',
 '18-NRV008',
 '18-NRV009',
 '18-NRV010',
 '18-NRV011',
 '18-NRV013',
 '18-NRV014',
 '18-NRV015',
 '18-NRV016',
 '18-NRV017',
 '18-NRV018',
 '18-NRV019',
 '18-NRV020',
 '18-NRV022',
 '18-NRV024',
 '18-NRV025',
 '18-NRV026',
 '18-NRV028',
 '18-NRV029',
 '18-NRV030',
 '18-NRV031',
 '18-NRV033',
 '18-NRV034',
 '18-NRV035',
 '18-NRV037',
 '18-NRV040',
 '18-NRV045',
 '18-NRV046',
 '18-NRV047',
 '18-NRV050',
 '18-NRV051',
 '18-NRV052',
 '18-NRV053',
 '18-NRV054',
 '18-NRV055',
 '18-NRV056',
 '18-NRV057',
 '18-NRV060',
 '18-NRV061',
 '18-NRV062',
 '18-NRV063',
 '18-NRV067',
 '18-NRV069',
 '18-NRV071',
 '18-SAN088',
 '18-SLC154',
 '18-SNB002',
 '19-10 BF',
 '19-11',
 '19-12',
 '19-2',
 '19-4',
 '19-5',
 '19-6',
 '19-7',
 '19-8',
 '2-BEAR',
 '2-BRE',
 '20-AXCSSD',
 '20-AXTSSD',
 '20-AXTU',
 '20-CENM',
 '20-EPH1M',
 '20-EPH2M',
 '20-EPH3M',
 '20-EPHU',
 '20-FAYM',
 '20-FRVM',
 '20-FRVU',
 '20-FTGM',
 '20-GUNM',
 '20-MAN1M',
 '20-MAN2M',
 '20-MANU',
 '20-MAYM',
 '20-MORM',
 '20-MORU',
 '20-MTP1M',
 '20-MTP1U',
 '20-MTP2M',
 '20-MTP2U',
 '20-MTP3U',
 '20-NSU',
 '20-SPRM',
 '20-SPRU',
 '20-SSU',
 '20-STRM',
 '20-WALM',
 '21-7',
 '22-PEOA15',
 '22-SCRL44',
 '22-SCRU8',
 '22-WAN17',
 '22-WEBC21',
 '23-EC-1',
 '23-EC-2',
 '23-EC-3',
 '23-EC-4',
 '23-EC-5',
 '23-ERD-01',
 '23-ERD-02',
 '23-G-1',
 '23-G-2',
 '23-G-3',
 '23-G-4',
 '23-G-5',
 '23-G-6',
 '23-LP-01',
 '23-LP-02',
 '25-DR02',
 '25-LE13S',
 '25-LE27',
 '25-SA06',
 '25-SF18',
 '27-COAND',
 '27-COUN01',
 '27-COUN02',
 '27-COUN03',
 '27-COUN04',
 '27-COUN05',
 '27-COUN06',
 '27-COUN07',
 '27-SGAIR',
 '29-TAY001',
 '29-TAY002',
 '29-VLY001',
 '29-VLY002',
 '29-VLY003',
 '29-VLY004',
 '29-WCW001',
 '29-WCW002',
 '3-LOG01',
 '3-LOG27',
 '3-LOG29',
 '3-UNINH',
 '3-UNINHC',
 '5-1002',
 '6-BO01',
 '6-BO02',
 '6-BO03',
 '6-BO04',
 '6-BO05',
 '6-BO06',
 '6-BO07',
 '6-BO08',
 '6-BO09',
 '6-BO10',
 '6-BO11',
 '6-BO12',
 '6-BO13',
 '6-BO14',
 '6-BO15',
 '6-BO16',
 '6-BO17',
 '6-BO18',
 '6-BO19',
 '6-BO20',
 '6-BO21',
 '6-BO22',
 '6-BO23',
 '6-BO24',
 '6-BO25',
 '6-BO26',
 '6-BO27',
 '6-BO28',
 '6-BO29',
 '6-BO30',
 '6-BO31',
 '6-BO32',
 '6-BO33',
 '6-CE01',
 '6-CE02',
 '6-CE03',
 '6-CE04',
 '6-CE05',
 '6-CE06',
 '6-CE07',
 '6-CE08',
 '6-CE09',
 '6-CE10',
 '6-CE11',
 '6-CE12',
 '6-CF01',
 '6-CF02',
 '6-CF03',
 '6-CF04',
 '6-CF05',
 '6-CF06',
 '6-CF07',
 '6-CF08',
 '6-CF09',
 '6-CF10',
 '6-CF11',
 '6-CF12',
 '6-CF13',
 '6-CF14',
 '6-CF15',
 '6-CF16',
 '6-CF17',
 '6-CL01',
 '6-CL02',
 '6-CL03',
 '6-CL04',
 '6-CL05',
 '6-CL06',
 '6-CL07',
 '6-CL08',
 '6-CL09',
 '6-CL10',
 '6-CL11',
 '6-CL12',
 '6-CL13',
 '6-DC01',
 '6-DC02',
 '6-DC03',
 '6-DC04',
 '6-FA01',
 '6-FA02',
 '6-FA03',
 '6-FA04',
 '6-FA05',
 '6-FA06',
 '6-FA07',
 '6-FA08',
 '6-FA09',
 '6-FA10',
 '6-FA11',
 '6-FA12',
 '6-FA13',
 '6-FA14',
 '6-FA15',
 '6-FA16',
 '6-FH01',
 '6-FH02',
 '6-FH03',
 '6-FH04',
 '6-FH05',
 '6-HA01',
 '6-HA02',
 '6-HA03',
 '6-KA01',
 '6-KA02',
 '6-KA03',
 '6-KA04',
 '6-KA05',
 '6-KA06',
 '6-KA07',
 '6-KA08',
 '6-KA09',
 '6-KA10',
 '6-KA11',
 '6-KA12',
 '6-KA13',
 '6-KA14',
 '6-KA15',
 '6-KA16',
 '6-KA17',
 '6-KA18',
 '6-KA19',
 '6-KA20',
 '6-KA21',
 '6-KA22',
 '6-LA01',
 '6-LA02',
 '6-LA03',
 '6-LA04',
 '6-LA05',
 '6-LA06',
 '6-LA07',
 '6-LA08',
 '6-LA09',
 '6-LA10',
 '6-LA11',
 '6-LA12',
 '6-LA13',
 '6-LA14',
 '6-LA15',
 '6-LA16',
 '6-LA17',
 '6-LA18',
 '6-LA19',
 '6-LA20',
 '6-LA21',
 '6-LA22',
 '6-LA23',
 '6-LA24',
 '6-LA25',
 '6-LA26',
 '6-LA27',
 '6-LA28',
 '6-LA29',
 '6-LA30',
 '6-LA31',
 '6-LA32',
 '6-LA33',
 '6-LA34',
 '6-LA35',
 '6-LA36',
 '6-LA37',
 '6-LA38',
 '6-LA39',
 '6-LA40',
 '6-LA41',
 '6-LA42',
 '6-LA43',
 '6-LA44',
 '6-LA45',
 '6-LA46',
 '6-LA47',
 '6-LA48',
 '6-NS01',
 '6-NS02',
 '6-NS03',
 '6-NS04',
 '6-NS05',
 '6-NS06',
 '6-NS07',
 '6-NS08',
 '6-NS09',
 '6-NS10',
 '6-NS11',
 '6-SU01',
 '6-SU02',
 '6-SU03',
 '6-SW01',
 '6-SW02',
 '6-SW03',
 '6-SW04',
 '6-SY01',
 '6-SY02',
 '6-SY03',
 '6-SY04',
 '6-SY05',
 '6-SY06',
 '6-SY07',
 '6-SY08',
 '6-SY09',
 '6-SY10',
 '6-SY11',
 '6-SY12',
 '6-SY13',
 '6-SY14',
 '6-SY15',
 '6-SY16',
 '6-SY17',
 '6-SY18',
 '6-SY19',
 '6-WB01',
 '6-WB02',
 '6-WB03',
 '6-WB04',
 '6-WP01',
 '6-WP02',
 '6-WP03',
 '6-WP04',
 '6-WP05',
 '6-WP06',
 '6-WP07',
 '6-WP08',
 '6-WX01',
 '6-WX02',
 '6-WX03',
 '6-WX04',
 '6-WX05',
 '6-WX06',
 '6-WX07',
 '7-AL11',
 '7-AL12',
 '7-AR11',
 '7-BB11',
 '7-BB21',
 '7-CE11',
 '7-DU11',
 '7-DU12',
 '7-DU21',
 '7-DU22',
 '7-FR11',
 '7-FR21',
 '7-MH11',
 '7-MY11',
 '7-MY12',
 '7-NE11',
 '7-NE21',
 '7-RO12',
 '7-RO21',
 '7-RO22',
 '7-RO32',
 '7-RO41',
 '7-RO42',
 '7-RO51',
 '7-RO52',
 '7-RO62',
 '7-RO71',
 '7-RO72',
 '7-RO81',
 '7-RO82',
 '7-RO91',
 '7-RO92',
 '7-TA11',
 '7-TA12',
 '8-13']

In [61]:
shp_ids_to_add = list(ut_shapes[ut_shapes["CountyID"].isin(["10","16","26","6","7"])]["shp_ID"])

remove = ["7-AL11","7-BB11","7-DU11","7-FR21","7-MY11","2-BEAR","23-ERD-02","16-1","19-10 BF","23-ERD","23-ERD-01"]


    

In [62]:
no_dissolve_list = no_dissolve_list+shp_ids_to_add

no_dissolve_list = [x for x in no_dissolve_list if (x not in remove)]

In [63]:
#Don't dissolve at all: 19-10 BF

In [64]:
ut_shapes_dissolve = ut_shapes[~ut_shapes["shp_ID"].isin(no_dissolve_list)]
ut_shapes_no_dissolve = ut_shapes[ut_shapes["shp_ID"].isin(no_dissolve_list)]

ut_shapes_no_dissolve["shp_ID"] = ut_shapes_no_dissolve["shp_ID"] +"-"+ut_shapes_no_dissolve["VistaID"]

/Users/peterhorton/opt/anaconda3/envs/geo_env/lib/python3.9/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [65]:
ut_shapes_dissolve = pd.concat([ut_shapes_no_dissolve,ut_shapes_dissolve])

In [66]:
print(ut_shapes.shape)
print(ut_shapes_dissolve.shape)
print(ut_shapes_no_dissolve.shape)

(3074, 15)
(3074, 15)
(749, 15)


In [67]:
ut_shapes_dissolved = ut_shapes_dissolve.dissolve(["shp_ID"])
ut_shapes_dissolved.reset_index(inplace=True,drop=False)

In [68]:
ut_shapes_dissolved.shape

(2441, 15)

In [69]:
#ut_shapes_dissolved = ut_shapes_dissolved[ut_shapes_no_dissolve.columns]

In [70]:
ut_shapes_combined = ut_shapes_dissolved

In [71]:
#ut_shapes_combined = gp.GeoDataFrame(pd.concat([ut_shapes_dissolved,ut_shapes_no_dissolve]))

In [72]:
source_join_one = pd.merge(election_results,ut_shapes_combined,on="shp_ID",how="outer",indicator=True)
source_join_one["_merge"].value_counts()

right_only    1460
left_only     1364
both           981
Name: _merge, dtype: int64

In [73]:
election_shp_ID_changes = pd.read_csv("./election_shp_ID_changes.csv")
election_shp_ID_changes_dict = dict(zip(election_shp_ID_changes["election_ID"],election_shp_ID_changes["shp_ID"]))

In [74]:
election_results["shp_ID"] = election_results["shp_ID"].map(election_shp_ID_changes_dict).fillna(election_results["shp_ID"])

<ipython-input-74-2339883d4d08>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  election_results["shp_ID"] = election_results["shp_ID"].map(election_shp_ID_changes_dict).fillna(election_results["shp_ID"])


In [75]:
source_join_two = pd.merge(election_results,ut_shapes_combined,on="shp_ID",how="outer",indicator=True)
source_join_two["_merge"].value_counts()

both          2289
right_only     152
left_only       56
Name: _merge, dtype: int64

In [76]:
source_join_two[source_join_two["_merge"]=="left_only"].to_csv("./election_results.csv")
source_join_two[source_join_two["_merge"]=="right_only"].to_csv("./ut_shapes_dissolved.csv")

In [77]:
pd.options.display.max_rows = 100

In [78]:
source_join_two[(source_join_two["_merge"]=="both")&(source_join_two["county"]=="Duchesne")].head(20)

,pivot_col,county,precinct,G20ATGDSKO,G20ATGLBAU,G20ATGRREY,G20AUDCOST,G20AUDRDOU,G20AUDUFAB,G20GOVADUE,G20GOVDPET,G20GOVLCOT,G20GOVOWRI,G20GOVRCOX,G20PRECBLA,G20PREDBID,G20PREGHAW,G20PREILAR,G20PREIMCH,G20PREIPIE,G20PREIWES,G20PRELJOR,G20PREOWRI,G20PRERTRU,G20TRECPRO,G20TRELSPE,G20TRERDAM,CountyID_x,unique_ID,shp_ID,geometry,OBJECTID,CountyID_y,VistaID,PrecinctID,SubPrecinc,VersionNbr,EffectiveD,AliasName,Comments,RcvdDate,GlobalID,SHAPE_Leng,SHAPE_Area,_merge
523,Duchesne-AL11UC - UPPER COMM REC SSD,Duchesne,AL11UC - UPPER COMM REC SSD,12.0,12.0,291.0,26.0,274.0,7.0,6.0,9.0,13.0,98.0,239.0,0.0,13.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,307.0,18.0,13.0,281.0,7,7-AL11UC - UPPER COMM REC SSD,7-AL11,"POLYGON ((-110.27415 40.21463, -110.27426 40.2...",1205.0,7,AL11,AL11,GW,4.0.3.18,2020-08-28,None,None,2020-08-28,{2D54DDAF-8CB5-4A91-B367-6176FE9F1FC6},0.427397,0.010421,both
524,Duchesne-AL12 Altamont - City,Duchesne,AL12 Altamont - City,7.0,3.0,91.0,5.0,89.0,1.0,1.0,6.0,3.0,18.0,82.0,0.0,7.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,93.0,8.0,3.0,84.0,7,7-AL12 Altamont - City,7-AL12-AL12,"POLYGON ((-110.28300 40.36302, -110.28299 40.3...",1149.0,7,AL12,AL12,None,4.0.3.18,2020-08-28,None,None,2020-08-28,{6B45AA64-D5BC-4B6C-B028-5B3F4C433FBD},0.038314,0.000058,both
525,Duchesne-AR11 Arcadia - County,Duchesne,AR11 Arcadia - County,12.0,6.0,65.0,6.0,67.0,5.0,1.0,15.0,3.0,20.0,55.0,0.0,16.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,65.0,5.0,10.0,65.0,7,7-AR11 Arcadia - County,7-AR11-AR11,"POLYGON ((-110.06742 40.20039, -110.07176 40.2...",203.0,7,AR11,AR11,None,4.0.3.18,2020-08-28,None,None,2020-08-28,{1EAD368C-99FF-4A51-9619-100F951A9CBF},0.310637,0.005197,both
526,Duchesne-AR11GW - GATEWAY SSD,Duchesne,AR11GW - GATEWAY SSD,2.0,1.0,55.0,5.0,53.0,0.0,0.0,1.0,2.0,20.0,45.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,59.0,6.0,0.0,51.0,7,7-AR11GW - GATEWAY SSD,7-AR11-AR11:GW,"POLYGON ((-110.16099 40.21471, -110.16089 40.2...",948.0,7,AR11:GW,AR11,GW,4.0.3.18,2020-08-28,None,None,2020-08-28,{DC12E21F-6AC3-4CD7-AA17-4C5FB6E4AA92},0.333683,0.004945,both
527,Duchesne-BB11UC - UPPER COMM REC SSD,Duchesne,BB11UC - UPPER COMM REC SSD,17.0,15.0,262.0,15.0,267.0,7.0,5.0,23.0,4.0,106.0,212.0,0.0,14.0,0.0,0.0,0.0,1.0,0.0,4.0,2.0,287.0,17.0,11.0,261.0,7,7-BB11UC - UPPER COMM REC SSD,7-BB11,"POLYGON ((-109.97762 40.49896, -109.97861 40.4...",260.0,7,BB11:UC,BB11,UC,4.0.3.18,2020-08-28,None,None,2020-08-28,{34D3FC21-EDA7-4CE4-A365-1586BCDBA716},1.572764,0.065914,both
528,Duchesne-BB21 Bluebell/Upalco - County,Duchesne,BB21 Bluebell/Upalco - County,0.0,0.0,8.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,8.0,7,7-BB21 Bluebell/Upalco - County,7-BB21-BB21,"POLYGON ((-110.12419 40.37393, -110.12419 40.3...",2231.0,7,BB21,BB21,None,4.0.3.18,2020-08-28,None,None,2020-08-28,{47CF299E-92C9-4B77-B5D9-B94D322D6D6A},0.185355,0.001401,both
529,Duchesne-BB21UC - UPPER COMM REC SSD,Duchesne,BB21UC - UPPER COMM REC SSD,7.0,4.0,121.0,11.0,115.0,5.0,3.0,6.0,4.0,30.0,103.0,0.0,4.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,126.0,8.0,5.0,116.0,7,7-BB21UC - UPPER COMM REC SSD,7-BB21-BB21:UC,"POLYGON ((-110.12415 40.34482, -110.12416 40.3...",194.0,7,BB21:UC,BB21,UC,4.0.3.18,2020-08-28,None,None,2020-08-28,{50887053-7639-40AA-A538-0F9AB55B8699},0.510090,0.008505,both
530,Duchesne-CE11 Cedarview - County,Duchesne,CE11 Cedarview - County,33.0,18.0,265.0,30.0,263.0,13.0,15.0,26.0,14.0,88.0,218.0,0.0,31.0,3.0,0.0,0.0,0.0,1.0,5.0,4.0,279.0,22.0,17.0,267.0,7,7-CE11 Cedarview - County,7-CE11-CE11,"POLYGON ((-109.97676 40.34194, -109.99192 40.3...",2665.0,7,CE11,CE11,None,4.0.3.18,2020-08-28,None,None,2020-08-28,{774660D6-28DE-4E9A-B7EF-E02C52155534},0.397188,0.005717,both
531,Duchesne-DU11GW - GATEWAY SSD,Duchesne,DU11GW - GATEWAY SSD,51.0,16.0,306.0,41.0,299.0,13.0,6.0,49.0,8.0,92.0,264.0,0.0,54.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,327.0,34.0,30.0,295.0,7,7-DU11GW - GATEWAY SSD,7-DU11,"POLYGON ((-110.24975 39.80662, -110.25073 39.8...",708.0,7,DU11:GW,DU11,GW,4.0.3.18,2020-08-28,None,None,2020-08-28,{9E4C5AFF-1

In [79]:
#Don't dissolve Grand precincts (10)
#Don't dissolve in this list ["12-EU","12-NE01","12-NE02","12-NE03","13-CW"]
#Don't dissolve in Plute (16)
#Don't dissolve Wasatch (26)
#Don't dissolve Davis (6)
#Don't dissolve Duchesne (7)